In [1]:
# jup2py.py
# program to convert functions in cells to indivdiual .py files
# removes magic commands and asyncio conflicting code

# NOTE: the first line of Module's cell to be converted should have the following:
# <function_name>.py

"""Program to convert .ipynb to .py files
Date: 07-Oct-2019
Ver: 1.0
"""

import json
from os import listdir

fs = listdir() # get the filenames

# make an exclusion list
magics = %lsmagic
magic_words = [s for s in str(magics).split(' ') if s not in '' if '%' in s[0]]
magic_words = [s for s in magic_words if s != '%']

ignores = ['ztest', 'jup2py'] # ignore filenames having these words

# exclude lines having these words and magic commands 
exclude = ['util.startLoop()', 'import nest_asyncio', 'nest_asyncio.apply()'] + magic_words

# remove unwanted file extensions
ipfilelist = [f for f in fs 
              if f[-5:] == 'ipynb' 
              if f[:1] not in ['_'] 
              if f in [x for x in fs 
                       if all(item not in x for item in ignores)] ]

for file in ipfilelist:
    code_cells = []  #initialize code_cells
    with open(file) as datafile:
        code = json.load(datafile)
        code_cells.append([cell['source'] for cell in code['cells'] if cell['cell_type'] == 'code'])
        codes = [cell for cells in code_cells for cell in cells if cell]
        code_dict = {pycode[0][2:-1]:pycode for pycode in codes if pycode[0][-4:] == '.py\n'}
        with open(file[:-6]+'.py', 'w') as f:
            for k, v in code_dict.items():
                for line in v:
                    if not any(word in line for word in exclude):
                        f.write(line)        
                f.write('\n\n#_____________________________________\n\n')